In [1]:
import numpy as np
import tensorflow as tf
import gym
from collections import deque

/Users/apple/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [58]:
# 하이퍼 파라미터
gridSize = 7
nbActions = gridSize * gridSize
nbStates = gridSize * gridSize

LEARNING_RATE = 0.005
INPUT = nbStates
OUTPUT = nbActions
DISCOUNT = 0.99

winReward = 1

STONE_NONE = 0
STONE_PLAYER1 = 1
STONE_PLAYER2 = -1
STONE_MAX = 5

In [57]:
class Omok:
    # --------------------------------
    # 초기화
    # --------------------------------
    def __init__(self, gridSize):
        self.gridSize = gridSize
        self.nbStates = self.gridSize * self.gridSize
        self.state = np.zeros(self.nbStates, dtype=np.uint8)
        
    # --------------------------------
    # 리셋
    # --------------------------------
    def reset(self):
        self.state = np.zeros(self.nbStates, dtype=np.uint8)
        
    # --------------------------------
    # 현재 상태 구함
    # --------------------------------
    def getState(self):
        return np.reshape(self.state, (1, self.nbStates))
    
    # --------------------------------
    # render
    # --------------------------------
    def render(self):
        state = np.reshape(self.state,(self.gridSize,self.gridSize))
        print(state)
        return state
    
    # --------------------------------
    # 매칭 검사
    # 다섯개 붙어 있으면 true, 아님 false
    # --------------------------------
    def CheckMatch(self, player):
        for y in range(self.gridSize):
            for x in range(self.gridSize):

                # --------------------------------
                # 오른쪽 검사
                # --------------------------------
                match = 0

                for i in range(STONE_MAX):
                    if (x + i >= self.gridSize):
                        break

                    if (self.state[y * self.gridSize + x + i] == player):
                        match += 1
                    else:
                        break;

                    if (match >= STONE_MAX):
                        return True

                # --------------------------------
                # 아래쪽 검사
                # --------------------------------
                match = 0

                for i in range(STONE_MAX):
                    if (y + i >= self.gridSize):
                        break

                    if (self.state[(y + i) * self.gridSize + x] == player):
                        match += 1
                    else:
                        break;

                    if (match >= STONE_MAX):
                        return True

                # --------------------------------
                # 오른쪽 대각선 검사
                # --------------------------------
                match = 0

                for i in range(STONE_MAX):
                    if ((x + i >= self.gridSize) or (y + i >= self.gridSize)):
                        break

                    if (self.state[(y + i) * self.gridSize + x + i] == player):
                        match += 1
                    else:
                        break;

                    if (match >= STONE_MAX):
                        return True

                # --------------------------------
                # 왼쪽 대각선 검사
                # --------------------------------
                match = 0

                for i in range(STONE_MAX):
                    if ((x - i < 0) or (y + i >= self.gridSize)):
                        break

                    if (self.state[(y + i) * self.gridSize + x - i] == player):
                        match += 1
                    else:
                        break;

                    if (match >= STONE_MAX):
                        return True

        return False
    
    # --------------------------------
    # 게임오버 검사
    # 리워드를 리턴
    # --------------------------------
    def getReward(self, player):
        if (self.CheckMatch(STONE_PLAYER1) == True):
            if (player == STONE_PLAYER1):
                return winReward
            else:
                return -winReward
        elif (self.CheckMatch(STONE_PLAYER2) == True):
            if (player == STONE_PLAYER1):
                return -winReward
            else:
                return winReward
        else:
            for i in range(self.nbStates):
                if (self.state[i] == STONE_NONE):
                    return 0
            return 0

        
    # --------------------------------
    # action취해서 state update
    # player : 다음 player로 전환
    # --------------------------------
    def getNextState(self, self.state, player, action):
        self.state[action] = player;
        player = -player
        
        return self.state, player